In [1]:
import numpy as np
import pandas as pd
import random
from typing import List, Tuple, Optional
import json

class MorpionDatasetGenerator:
    def __init__(self):
        self.VIDE = 0
        self.JOUEUR_X = 1
        self.JOUEUR_O = 2
        self.positions_visitees = set()
        self.dataset = []
    
    def afficher_plateau(self, plateau):
        """Affiche le plateau de manière lisible"""
        symboles = {0: '.', 1: 'X', 2: 'O'}
        for i in range(3):
            print(' '.join(symboles[plateau[i*3 + j]] for j in range(3)))
        print()
    
    def verifier_victoire(self, plateau, joueur):
        """Vérifie si le joueur a gagné"""
        # Lignes
        for i in range(3):
            if all(plateau[i*3 + j] == joueur for j in range(3)):
                return True
        
        # Colonnes
        for j in range(3):
            if all(plateau[i*3 + j] == joueur for i in range(3)):
                return True
        
        # Diagonales
        if all(plateau[i*3 + i] == joueur for i in range(3)):
            return True
        if all(plateau[i*3 + (2-i)] == joueur for i in range(3)):
            return True
        
        return False
    
    def plateau_plein(self, plateau):
        """Vérifie si le plateau est plein"""
        return all(case != self.VIDE for case in plateau)
    
    def evaluer_position(self, plateau):
        """Évalue la position finale du plateau"""
        if self.verifier_victoire(plateau, self.JOUEUR_X):
            return 10
        elif self.verifier_victoire(plateau, self.JOUEUR_O):
            return -10
        else:
            return 0
    
    def coups_possibles(self, plateau):
        """Retourne la liste des coups possibles"""
        return [i for i in range(9) if plateau[i] == self.VIDE]
    
    def minimax_alpha_beta(self, plateau, profondeur, alpha, beta, maximisant):
        """Algorithme Minimax avec élagage Alpha-Beta"""
        # Vérifier les conditions de fin
        if self.verifier_victoire(plateau, self.JOUEUR_X):
            return 10 - profondeur
        elif self.verifier_victoire(plateau, self.JOUEUR_O):
            return -10 + profondeur
        elif self.plateau_plein(plateau):
            return 0
        
        coups = self.coups_possibles(plateau)
        
        if maximisant:  # Tour du joueur X
            max_eval = float('-inf')
            for coup in coups:
                plateau[coup] = self.JOUEUR_X
                eval_score = self.minimax_alpha_beta(plateau, profondeur + 1, alpha, beta, False)
                plateau[coup] = self.VIDE
                max_eval = max(max_eval, eval_score)
                alpha = max(alpha, eval_score)
                if beta <= alpha:
                    break  # Élagage Alpha-Beta
            return max_eval
        else:  # Tour du joueur O
            min_eval = float('inf')
            for coup in coups:
                plateau[coup] = self.JOUEUR_O
                eval_score = self.minimax_alpha_beta(plateau, profondeur + 1, alpha, beta, True)
                plateau[coup] = self.VIDE
                min_eval = min(min_eval, eval_score)
                beta = min(beta, eval_score)
                if beta <= alpha:
                    break  # Élagage Alpha-Beta
            return min_eval
    
    def meilleur_coup(self, plateau, joueur):
        """Trouve le meilleur coup pour le joueur donné"""
        coups = self.coups_possibles(plateau)
        if not coups:
            return None
        
        meilleur_score = float('-inf') if joueur == self.JOUEUR_X else float('inf')
        meilleur_mouvement = coups[0]
        
        for coup in coups:
            plateau[coup] = joueur
            if joueur == self.JOUEUR_X:
                score = self.minimax_alpha_beta(plateau, 0, float('-inf'), float('inf'), False)
                if score > meilleur_score:
                    meilleur_score = score
                    meilleur_mouvement = coup
            else:
                score = self.minimax_alpha_beta(plateau, 0, float('-inf'), float('inf'), True)
                if score < meilleur_score:
                    meilleur_score = score
                    meilleur_mouvement = coup
            plateau[coup] = self.VIDE
        
        return meilleur_mouvement, meilleur_score
    
    def generer_toutes_positions(self, plateau, joueur_actuel):
        """Génère récursivement toutes les positions possibles"""
        plateau_str = ''.join(map(str, plateau))
        
        # Éviter les doublons
        if plateau_str in self.positions_visitees:
            return
        self.positions_visitees.add(plateau_str)
        
        # Vérifier si la partie est terminée
        if (self.verifier_victoire(plateau, self.JOUEUR_X) or 
            self.verifier_victoire(plateau, self.JOUEUR_O) or 
            self.plateau_plein(plateau)):
            return
        
        # Ajouter la position actuelle au dataset
        coups = self.coups_possibles(plateau)
        if coups:
            meilleur_coup_info = self.meilleur_coup(plateau, joueur_actuel)
            if meilleur_coup_info:
                meilleur_coup_pos, score = meilleur_coup_info
                
                # Normaliser le score entre -1 et 1
                score_normalise = max(-1, min(1, score / 10))
                
                self.dataset.append({
                    'position': plateau.copy(),
                    'joueur_actuel': joueur_actuel,
                    'meilleur_coup': meilleur_coup_pos,
                    'score': score_normalise,
                    'coups_possibles': coups.copy()
                })
        
        # Générer récursivement les positions suivantes
        for coup in coups:
            plateau[coup] = joueur_actuel
            prochain_joueur = self.JOUEUR_O if joueur_actuel == self.JOUEUR_X else self.JOUEUR_X
            self.generer_toutes_positions(plateau, prochain_joueur)
            plateau[coup] = self.VIDE
    
    def generer_parties_aleatoires(self, nb_parties=1000):
        """Génère des parties aléatoires pour enrichir le dataset"""
        print(f"Génération de {nb_parties} parties aléatoires...")
        
        for partie in range(nb_parties):
            plateau = [self.VIDE] * 9
            joueur_actuel = random.choice([self.JOUEUR_X, self.JOUEUR_O])
            
            while True:
                coups = self.coups_possibles(plateau)
                if not coups:
                    break
                
                if (self.verifier_victoire(plateau, self.JOUEUR_X) or 
                    self.verifier_victoire(plateau, self.JOUEUR_O)):
                    break
                
                # 70% de chance de jouer le meilleur coup, 30% de chance de jouer aléatoirement
                if random.random() < 0.7:
                    meilleur_coup_info = self.meilleur_coup(plateau, joueur_actuel)
                    if meilleur_coup_info:
                        coup, score = meilleur_coup_info
                        score_normalise = max(-1, min(1, score / 10))
                        
                        plateau_str = ''.join(map(str, plateau))
                        if plateau_str not in self.positions_visitees:
                            self.positions_visitees.add(plateau_str)
                            self.dataset.append({
                                'position': plateau.copy(),
                                'joueur_actuel': joueur_actuel,
                                'meilleur_coup': coup,
                                'score': score_normalise,
                                'coups_possibles': coups.copy()
                            })
                        
                        plateau[coup] = joueur_actuel
                else:
                    coup = random.choice(coups)
                    plateau[coup] = joueur_actuel
                
                joueur_actuel = self.JOUEUR_O if joueur_actuel == self.JOUEUR_X else self.JOUEUR_X
    
    def generer_dataset_complet(self, inclure_parties_aleatoires=True):
        """Génère le dataset complet"""
        print("Génération de toutes les positions optimales...")
        
        # Générer toutes les positions possibles
        plateau_vide = [self.VIDE] * 9
        self.generer_toutes_positions(plateau_vide, self.JOUEUR_X)
        
        positions_optimales = len(self.dataset)
        print(f"Positions optimales générées: {positions_optimales}")
        
        # Ajouter des parties aléatoires pour la diversité
        if inclure_parties_aleatoires:
            self.generer_parties_aleatoires(2000)
        
        print(f"Dataset total: {len(self.dataset)} exemples")
        return self.dataset
    
    def sauvegarder_dataset(self, nom_fichier="morpion_dataset"):
        """Sauvegarde le dataset dans différents formats"""
        if not self.dataset:
            print("Aucun dataset à sauvegarder!")
            return
        
        # Préparer les données pour la sauvegarde
        data_pour_csv = []
        for exemple in self.dataset:
            row = {}
            # Ajouter les 9 positions du plateau
            for i in range(9):
                row[f'pos_{i}'] = exemple['position'][i]
            
            row['joueur_actuel'] = exemple['joueur_actuel']
            row['meilleur_coup'] = exemple['meilleur_coup']
            row['score'] = exemple['score']
            row['nb_coups_possibles'] = len(exemple['coups_possibles'])
            
            data_pour_csv.append(row)
        
        # Sauvegarder en CSV
        df = pd.DataFrame(data_pour_csv)
        df.to_csv(f"{nom_fichier}.csv", index=False)
        print(f"Dataset sauvegardé: {nom_fichier}.csv ({len(df)} exemples)")
        
        # Sauvegarder en JSON (plus complet)
        with open(f"{nom_fichier}.json", 'w') as f:
            json.dump(self.dataset, f, indent=2)
        print(f"Dataset sauvegardé: {nom_fichier}.json")
        
        # Afficher quelques statistiques
        self.afficher_statistiques()
    
    def afficher_statistiques(self):
        """Affiche les statistiques du dataset"""
        if not self.dataset:
            return
        
        print("\n=== STATISTIQUES DU DATASET ===")
        print(f"Nombre total d'exemples: {len(self.dataset)}")
        
        # Répartition par joueur
        exemples_x = sum(1 for ex in self.dataset if ex['joueur_actuel'] == self.JOUEUR_X)
        exemples_o = sum(1 for ex in self.dataset if ex['joueur_actuel'] == self.JOUEUR_O)
        print(f"Exemples pour joueur X: {exemples_x}")
        print(f"Exemples pour joueur O: {exemples_o}")
        
        # Répartition des scores
        scores = [ex['score'] for ex in self.dataset]
        print(f"Score moyen: {np.mean(scores):.3f}")
        print(f"Score min: {min(scores):.3f}")
        print(f"Score max: {max(scores):.3f}")
        
        # Exemple de quelques positions
        print("\n=== EXEMPLES DE POSITIONS ===")
        for i, exemple in enumerate(self.dataset[:3]):
            print(f"\nExemple {i+1}:")
            print(f"Joueur actuel: {'X' if exemple['joueur_actuel'] == 1 else 'O'}")
            print(f"Meilleur coup: position {exemple['meilleur_coup']}")
            print(f"Score: {exemple['score']:.3f}")
            print("Plateau:")
            self.afficher_plateau(exemple['position'])

# Utilisation
if __name__ == "__main__":
    # Créer le générateur
    generateur = MorpionDatasetGenerator()
    
    # Générer le dataset complet
    dataset = generateur.generer_dataset_complet()
    
    # Sauvegarder
    generateur.sauvegarder_dataset("morpion_training_data")
    
    print("\nDataset généré avec succès!")
    print("Vous pouvez maintenant utiliser 'morpion_training_data.csv' pour entraîner votre modèle ML.")

Génération de toutes les positions optimales...
Positions optimales générées: 4520
Génération de 2000 parties aléatoires...
Dataset total: 4882 exemples
Dataset sauvegardé: morpion_training_data.csv (4882 exemples)
Dataset sauvegardé: morpion_training_data.json

=== STATISTIQUES DU DATASET ===
Nombre total d'exemples: 4882
Exemples pour joueur X: 2785
Exemples pour joueur O: 2097
Score moyen: 0.229
Score min: -1.000
Score max: 1.000

=== EXEMPLES DE POSITIONS ===

Exemple 1:
Joueur actuel: X
Meilleur coup: position 0
Score: 0.000
Plateau:
. . .
. . .
. . .


Exemple 2:
Joueur actuel: O
Meilleur coup: position 4
Score: 0.000
Plateau:
X . .
. . .
. . .


Exemple 3:
Joueur actuel: X
Meilleur coup: position 3
Score: 0.600
Plateau:
X O .
. . .
. . .


Dataset généré avec succès!
Vous pouvez maintenant utiliser 'morpion_training_data.csv' pour entraîner votre modèle ML.
